# Climate change and multilateral migration

This code: *Hendrik Scheewel* (hendrik.scheewel@uliege.be)

Todo:
- [ ] Compare population levels by country
- [ ] Compare fractions of people in cities by country
- [ ] Compare internal and international migration

# Load packages

In [ ]:
# import packages
import pandas as pd
import numpy as np
import math
import datetime
import pycountry
from numba import jit, vectorize, float64, guvectorize, njit
import matplotlib.pyplot as plt
from IPython.core.display import HTML
import seaborn as sns
sns.set_palette('Paired')
import itertools
from scipy.optimize import minimize
import statsmodels.formula.api as sm
colors = sns.color_palette()

In [ ]:
R = ('a', 'n')                      # regions
S = ('l', 'h')                      # skills
BB = ('d', 'f')                     # areas
T = (1980, 2010, 2040, 2070, 2100)  # periods of interest
t0 = 1950                           # first period

# Simulation class

In [ ]:
class model:
    def __init__(self,
                 name, # name of the model instance
                 σ={'a': math.inf, 'n': 2.0}, # elasticity of substitution
                 θ=0.25, # preference for children (NOT NEEDED?)
                 λ=0.5, # elasticity of training technology (NOT NEEDED?)
                 φ=0.14, # time to raise a child (NOT NEEDED?)
                 μ=1.4, # 1/elasticity of mig. to wages
                 B=0.5, # income loss due to forced discplacement
                 τ={'a': 0.0, 'n': 0.0}, # utility loss due to CLC
                 tol=1e-10, # tolerance parameter
                 data='data.xlsx', # input of data
                 scenario='intermediate'): # climate change scenario
        self.σ, self.θ, self.λ, self.φ, = σ, θ, λ, φ, 
        self.τ, self.μ, self.B  = τ, μ, B
        self.tol = tol
        self.name = name
        
        def get_input(which, dim): 
            ''' Function that unpacks inputs of the model '''
            return pd.read_excel(
            'input/%s' % (which),
            sheet_name=dim,
            index_col=0,
            header=list(range(len(dim))))

        # Load country-specific data
        # i = country-specific
        data_i = get_input(data, 'i')
        # irt  = country-skill-time-specific
        data_ist = get_input(data, 'ist')  
        # irt  = country-region-time-specific
        data_irt = get_input(data, 'irt')  
        # irst = country-region-skill-time-specific
        data_irst = get_input(data, 'irst')  
        # irsbt = country-region-skill-area-time-specific
        data_irsbt = get_input(data, 'irsbt')  
        # irsbt = origin-destination-region-skill-time-specific
        data_ijrst = get_input(data, 'ijrsbt') 
        
        # Assign country-specific data
        self.countries = data_i.index
        self.OECD = data_i['OECD'] # OECD indicator
        self.iso = data_i['iso3'] # iso3 Code
        self.Γ__m = data_ist['Γ__m'] # 
        self.A = data_irt['A'] # total factor productivity
        self.A_epsilon = data_irt['A_epsilon'] # total factor productivity used for ε calib
        self.ω = data_irt['ω'] # 
        self.Γ__n = data_irt['Γ__n'] # high-to-low-skill fertility ratio
        self.Γ__w = data_irt['Γ__w'] # high-to-low-skill wage ratio
        self.L = data_irst['L'] # resident population
        self.N = data_irst['N'] # native population
        self.mii = data_irsbt['mii'] # emigrant to stayer ratio (internal)
        self.Mij = data_ijrst['Mij'] # number of emigrants
        self.n = data_irsbt['n']
        self.p = data_irsbt['p']
        
        
        # Climate change damage scenario
        self.D = pd.read_excel('input/%s' % (data),
            sheet_name='scenarios',
            index_col=0,
            header=[0,1,2,3])[scenario]['D']

In [ ]:
def correct_south_south(self):
    for s in S:
        self.L[('n',s,2010)] = self.L[('n',s,2010)] + (self.Mij[('a',s,'d',2010)] + self.Mij[('n',s,'d',2010)]).multiply(1-self.OECD,1).sum()
        
model.correct_south_south = correct_south_south

In [ ]:
def MidxDataFrame(self,dim,value=np.nan):
    """ Creates multi-indexed empty dataframe with country-index"""
    return(pd.DataFrame(value, index=self.countries,
                        columns=pd.MultiIndex.from_product(dim)))

model.MidxDataFrame = MidxDataFrame

## Calibration methods

### Calibration of aggregate externality ε

In [ ]:
def calib_epsilon(self):
    self.ε = {'a': np.nan, 'n': np.nan} # aggregate externality
    self.Γ__L = self.MidxDataFrame([R, [1980,2010]]) # high-to-low-skill resid. pop. ratio
    self.A_bar = pd.DataFrame(index=self.countries) # scale factor in TFP

    t = 1980
    for r in R:
        self.D[(r, t)] = self.D[(r, t+30)]
    
    for r in R:
        for t in [1980,2010]:
            self.Γ__L[(r,t)] = self.L[r,'h',t]/self.L[r,'l',t]
        

    df = pd.concat([self.A,self.Γ__L],1,keys=['A','Γ__L'])

    def reg(r):
        df = self.A[r].melt(var_name='year1980',value_name='A')
        df = df.join(self.Γ__L[r].melt(var_name='year1980'
                                       ,value_name='Γ__L')['Γ__L'])
        df.year1980 = np.where(df.year1980 == 1980,1,0)
        
        reg = sm.ols(formula="np.log(A) ~ np.log(Γ__L) + year1980", 
                     data=df).fit()

        return(reg.params)

    self.γ = reg('a')['year1980'] ** (1/30)
    self.γ = 1.00256317290118    

    t = 1980
    for r in R:
        self.ε[r] = 0.5 * reg(r)['np.log(Γ__L)']
        self.A_bar[r] = self.A[(r,t)]/(self.γ * self.Γ__L[(r,t)]**self.ε[r] * self.D[(r,t)])
    
model.calib_epsilon = calib_epsilon

### Calibration of skill-biased externality κ

In [ ]:
def calib_kappa(self):
    self.κ = {'a': 0.0, 'n': np.nan}
    self.Γ__η_bar = {'a': 1.32558139534884, 'n': np.nan}
    self.Γ__L = self.MidxDataFrame([R, [2010]])
    self.Γ__η = self.MidxDataFrame([['n'], [2010]])

    for r in R:
        self.Γ__L[(r,2010)] = self.L[(r,'h',2010)]/self.L[(r,'l',2010)]

    self.Γ__η[('n',2010)] = self.Γ__w[('n',2010)] * self.Γ__L[('n',2010)]**(1/self.σ['n'])

    df = pd.DataFrame({'Γ__η': self.Γ__η[('n',2010)],'Γ__L': self.Γ__L[('n',2010)]})

    reg = sm.ols(formula="np.log(Γ__η) ~ np.log(Γ__L)", data=df).fit()

    self.κ['n'] = 0.5*reg.params['np.log(Γ__L)']

    self.Γ__η_bar['n'] = np.exp(reg.params['Intercept'])

model.calib_kappa = calib_kappa

### Calibration of migration costs

In [ ]:
def calib_migcosts(self):
    σ, θ, λ, φ, μ, B = self.σ, self.θ, self.λ, self.φ, self.μ, self.B
    τ, ε, κ, Γ__η_bar = self.τ, self.ε, self.κ, self.Γ__η_bar
    tol = self.tol
    γ = self.γ
    
    t = 2010
    
    # Create empty dataframes for emigrant to stayer ratios, stayers and migrants
    self.miF = self.MidxDataFrame([R, S, [t]])
    self.Ms = self.MidxDataFrame([R, S, BB, [t]])
    self.Mii = self.MidxDataFrame([R, S, BB, [t]])
    self.MiF = self.MidxDataFrame([R, S, [t]])

    for s in S:
        N_s_hat     = (self.L[('a',s,t)] + self.L[('n',s,t)] 
                                 + (self.Mij[('a',s,'d',2010)] + self.Mij[('n',s,'d',2010)]).sum(1) 
                                 - (self.Mij[('a',s,'d',2010)] + self.Mij[('n',s,'d',2010)]).sum(0))
        L_a_s_hat   = self.L[('a',s,2010)]
        L_n_s_hat   = self.L[('n',s,2010)]
        M_aF_s_hat  = self.Mij[('a',s,'d',2010)].sum(1)
        M_nF_s_hat  = self.Mij[('n',s,'d',2010)].sum(1)
        I_s_hat     = (self.Mij[('a',s,'d',2010)] + self.Mij[('n',s,'d',2010)]).sum(0)
        m_an_s      = self.mii[('a',s,'d',2010)]
        v_n_s       = 1

        # for all countries: internal emigrant-to-stayer ratios of high-skilled = 0.3
        if s == 'h':
            m_an_s = pd.Series(0.3,index=self.countries)
        # problematic observations: set internal emigrant to stayer ratio = 0
        if s == 'l':
            m_an_s.loc['Tonga'] = 0

        m_aF_s = M_aF_s_hat/(L_a_s_hat - (1-v_n_s)*I_s_hat)
        m_nF_s = M_nF_s_hat/(L_n_s_hat - m_an_s * (L_a_s_hat - (1-v_n_s)*I_s_hat) - v_n_s * I_s_hat)
        N_a_s = (L_a_s_hat - (1-v_n_s)*I_s_hat) * (1+m_an_s+m_aF_s)
        N_n_s = (L_n_s_hat - m_an_s * (L_a_s_hat - (1-v_n_s)*I_s_hat) - v_n_s * I_s_hat) * (1 + m_nF_s)
        M_aa_s = N_a_s/(1 + m_an_s + m_aF_s)
        M_nn_s = N_n_s/(1 + m_nF_s)
        M_an_s = m_an_s * M_aa_s
        M_aF_s = m_aF_s * M_aa_s
        M_nF_s = m_nF_s * M_nn_s
        RES_a = abs(N_a_s - (M_aa_s+M_aF_s+M_an_s))
        RES_n = abs(N_n_s - (M_nn_s+M_nF_s))

        self.mii[('a',s,'d',t)]  = m_an_s
        self.mii[('n',s,'d',t)]  = 0
        self.miF[('a',s,t)]  = m_aF_s
        self.miF[('n',s,t)]  = m_nF_s
        self.N[('a',s,t)]    = N_a_s
        self.N[('n',s,t)]    = N_n_s
        self.Ms[('a',s,'d', t)]   = M_aa_s
        self.Ms[('n',s,'d', t)]   = M_nn_s
        self.Mii[('a',s,'d', t)]  = M_an_s
        self.Mii[('n',s,'d', t)]  = 0
        self.MiF[('a',s,t)]  = M_aF_s
        self.MiF[('n',s,t)]  = M_nF_s
        
    self.mij = self.MidxDataFrame([R, S, [t], list(self.countries)])
    
    for r in R:
        for s in S:
            self.mij.loc[:,(r,s)] = np.array(self.Mij[(r,s,'d',t)].multiply((1/self.Ms[(r,s,'d',t)]),0))
            
    
    self.Γ__L = self.MidxDataFrame([R, [t]])
    self.η = self.MidxDataFrame([R, S, [t]])
    self.Γ__η = self.MidxDataFrame([R, [t]])
    self.Γ__w = self.MidxDataFrame([R, [t]])
    self.Y = self.MidxDataFrame([R, [t]])
    self.w = self.MidxDataFrame([R, S, [t]])
    self.c = self.MidxDataFrame([R, S, BB, [t]])
    self.v = self.MidxDataFrame([R, S, BB, T])
    self.xii = self.MidxDataFrame([R, S])
    self.xij = self.MidxDataFrame([R, S, list(self.countries)])


    ''' Functions '''
    # Before we can pin down bilateral migration costs
    # we need to identify indirect utilities:

    def Γ__L_cal(r, t):
        ''' Skill ratio in the labor force '''
        return (self.L[(r, 'h', t)] / self.L[(r, 'l', t)])

    def Γ__η_cal(r, t):
        ''' Skill bias in relative productivity '''
        return (Γ__η_bar[r] * self.Γ__L[(r, t)]**κ[r])

    def η_cal(r, s, t):
        ''' Relative productivity '''
        η_h = self.Γ__η[(r, t)] / (1 + self.Γ__η[(r, t)])
        if s == 'h':
            return (η_h)
        else:
            return (1 - η_h)

    def A_cal(r, t):
        ''' Total factor productivity '''
        return (γ**(t - t0) * self.D[(r, t)]
                * self.A_bar[r] * self.Γ__L[(r, t)]**ε[r])

    def Y_cal(r, t):
        ''' Gross domestic product '''
        if r == 'n':
            return(self.A[(r, t)] 
                   * (self.η[(r, 'l', t)] * self.L[(r, 'l', t)]**((σ[r]-1)/σ[r])
                      + self.η[(r, 'h', t)] * self.L[(r, 'h', t)]**((σ[r]-1)/σ[r]))**(σ[r]/(σ[r]-1)))
        else:
            return(self.A[(r, t)] * (self.η[(r, 'l', t)] * self.L[(r, 'l', t)]
                                     + self.η[(r, 'h', t)] * self.L[(r, 'h', t)]))

    def w_cal(r, s, t):
        ''' Wage rate '''
        if r == 'a':
            return (η_cal(r, s, t) * self.A[(r, t)])
        if r == 'n':
            return (η_cal(r, s, t) * self.A[(r, t)] ** ((σ[r]-1)/σ[r]) 
                    * (Y_cal(r, t) / self.L[(r, s, t)])**(1 / σ[r]))

    def Γ__w_cal(r, t):
        ''' High-low-skilled wage ratio '''
        return(self.w[(r, 'h', t)]/self.w[(r, 'l', t)])

    def c_cal(r, s, b, t):
        ''' Consumption '''
        B = self.B
        if b == 'd':
            B = 0
        return((1-B) * self.w[(r, s, t)] )

    
    def v_cal(r, s, b, t):
        ''' Inner utility '''
        return((1 - τ[r]) * self.c[(r, s, b, t)])
    
    
    def xii_cal(r, s):
        ''' internal migration cost'''
        return(1 - self.mii[(r,s,b,t)]**μ * (self.v[(r,s,'d',t)]/self.v[('n',s,'d',t)]))
    
    def xij_cal(r, s):
        '''international migration cost'''
        vorig_over_vdest = np.outer(self.v[(r,s,'d',2010)],1/self.v[('n',s,'d',2010)])\
                            *(1-np.diag(np.ones(len(self.countries))))
        return(1 - self.mij[(r,s,t)]**self.μ
               * pd.DataFrame(vorig_over_vdest,index=self.countries,columns=self.countries))
    
    b = 'd'
    for t in [1980,2010]:
        for r in R:
            self.D[(r, t)] = self.D[(r, t+30)]
            self.Γ__L[(r, t)] = Γ__L_cal(r, t)
            self.Γ__η[(r, t)] = Γ__η_cal(r, t)
        for r in R:
            for s in S:
                self.η[(r, s, t)] = η_cal(r, s, t)
            self.Y[(r, t)] = Y_cal(r, t)
        for r in R:
            for s in S:
                self.w[(r, s, t)] = w_cal(r, s, t)
        for r in R:
            self.Γ__w[(r, t)] = Γ__w_cal(r, t)
        for r in R:
            for s in S:
                self.c[(r, s, b, t)] = c_cal(r, s, b, t)
        for r in R:
            for s in S:
                self.v[(r, s, b, t)] = v_cal(r, s, b, t)
    t = 2010
    for r in R:
        for s in S:
            self.xii[(r, s)] = xii_cal(r,s)
            self.xij.loc[:,(r,s)] = np.array(xij_cal(r,s))
            
    # Makes sure that we have no xii or xij < 0 (needs to be checked why this occurs)
    self.xii = np.maximum(self.xii,0)
    self.xij = np.maximum(self.xij,0)
            
    Mijf = self.MidxDataFrame([R, S, ['f'], 
                           [2010],list(self.countries)]).fillna(0)
    
    self.Mij = self.Mij.join(Mijf)
    
    for r in R:
        for s in S:
            self.Mii[(r,s,'f',2010)] = 0
    
        
model.calib_migcosts = calib_migcosts

### Calibration of population growth

In [ ]:
def calib_pop_growth(self):
    σ, θ, λ, φ, μ, B = self.σ, self.θ, self.λ, self.φ, self.μ, self.B
    τ, ε, κ, Γ__η_bar = self.τ, self.ε, self.κ, self.Γ__η_bar
    tol = self.tol
    γ = self.γ
    
    t = 1980
    
    self.p = self.MidxDataFrame([R, S, BB, [t]])
    self.n = self.MidxDataFrame([R, S, BB, [t]])

    ''' Functions '''
    
    def n_cal(r, s, b, t):
        ''' Calibrated fertilitiy '''
        if s == 'h':
            return (self.Γ__n[(r, t)]*self.n[(r,'l',b,t)])
        else:
            return ((self.N[(r,'l',t+30)] + self.N[(r,'h',t+30)])\
                    / (self.L[(r,'l',t)] 
                       + self.Γ__n[(r, t)] * self.L[(r,'h',t)]\
                    * (1 - 2 * p_cal(r, 'h', b, t))))
        
    def p_cal(r, s, b, t):
        if s == 'h':
            pmin, pmax = 0.5, 0.8
            wmin = self.w[(r,s,t)].min()
            wmax = self.w[(r,s,t)].max()
            return(pmax-(pmax-pmin)*(wmax-self.w[(r,s,t)])/(wmax-wmin))
        else:
            return(self.N[(r,'h',t+30)]/(self.n[(r,'l','d',t)] 
                                         * self.L[(r,'l',t)])
                  - p_cal(r, 'h', b, t) * self.Γ__n[(r, t)] 
                   * (self.L[(r,'h',t)]/self.L[(r,'l',t)]))
    
    t, b = 1980, 'd'
    for r in R:
        for s in S:
            self.n[(r,s,b,t)] = n_cal(r, s, b, t)
            self.p[(r,s,b,t)] = p_cal(r, s, b, t)

    for t in [2010,2040,2070,2100]:
        for r in R:
            for s in S:
                for b in BB:
                    self.n[(r,s,b,t)] = self.n[(r,s,'d',1980)]
                    self.p[(r,s,b,t)] = self.p[(r,s,'d',1980)]      

model.calib_pop_growth = calib_pop_growth

In [ ]:
def fertility(self):

    for t in [2010,2040,2070,2100]:
        for r in R:
            for s in S:
                for b in BB:
                    self.n[(r,s,b,t)] = self.n[(r,s,'d',t)]
                    self.p[(r,s,b,t)] = self.p[(r,s,'d',t)]      

model.fertility = fertility

Use UN projection for $L_{2040}$, $\Rightarrow w_{2040} \quad \Rightarrow v_{2040} \quad \Rightarrow m_{2040} \quad \Rightarrow N_{2040} \quad \Rightarrow p_{2010}, n_{2010}$.

Then estimate convergence: $\ln \left(\frac{x_{r,s,t+1}}{x_{r,s,t}}\right) = \alpha_{r,s} + \beta_{r,s} \ln \left(\frac{x_{r,s,t}^{USA}}{x_{r,s,t}}\right) + \gamma_{r,s} \left(\ln \left(\frac{x_{r,s,t}^{USA}}{x_{r,s,t}}\right)\right)^2$, where $x = n,p$

## Simulation method

In [ ]:
def simulate(self, max_iter=150, report=False, report_from=0):
    """ Load  parameters """
    σ, θ, λ, φ, μ, B = self.σ, self.θ, self.λ, self.φ, self.μ, self.B
    τ, ε, κ, Γ__η_bar = self.τ, self.ε, self.κ, self.Γ__η_bar
    tol = self.tol
    γ = self.γ

    A_bar, xii, xij = self.A_bar, self.xii, self.xij

    t = 1980
    # Create empty dataframes
    #self.N = self.MidxDataFrame([R, S, [t]])
    self.mii = self.MidxDataFrame([R, S, BB, [t]])
    self.mij = self.MidxDataFrame([R, S, BB, list(T), list(self.countries)])
    self.Ms = self.MidxDataFrame([R, S, BB, [2010]])
    self.Mij = self.MidxDataFrame([R, S, BB, list(T), list(self.countries)])
    self.Iii = self.MidxDataFrame([R, S, [t]])
    self.Iij = self.MidxDataFrame([R, S, [t]])
    
    """ Functions """

    def Γ__L_fun(r, t):
        ''' Skill ratio in the labor force '''
        return (self.L[(r, 'h', t)] / self.L[(r, 'l', t)])

    def Γ__η_fun(r, t):
        ''' Skill bias in relative productivity '''
        return (Γ__η_bar[r] * self.Γ__L[(r, t)]**κ[r])

    def η_fun(r, s, t):
        ''' Relative productivity '''
        η_h = self.Γ__η[(r, t)] / (1 + self.Γ__η[(r, t)])
        if s == 'h':
            return (η_h)
        else:
            return (1 - η_h)

    def A_fun(r, t):
        ''' Total factor productivity '''
        return (γ**(t - t0) * self.D[(r, t)]
                * self.A_bar[r] * self.Γ__L[(r, t)]**ε[r])

    def Y_fun(r, t):
        ''' Gross domestic product '''
        if r == 'n':
            return(self.A[(r, t)] * (self.η[(r, 'l', t)] * self.L[(r, 'l', t)]**((σ[r]-1)/σ[r])
                                     + self.η[(r, 'h', t)] * self.L[(r, 'h', t)]**((σ[r]-1)/σ[r]))**(σ[r]/(σ[r]-1)))
        else:
            return(self.A[(r, t)] * (self.η[(r, 'l', t)] * self.L[(r, 'l', t)]
                                     + self.η[(r, 'h', t)] * self.L[(r, 'h', t)]))

    def w_fun(r, s, t):
        ''' Wage rate '''
        if r == 'a':
            return (η_fun(r, s, t) * self.A[(r, t)])
        if r == 'n':
            return (η_fun(r, s, t) * self.A[(r, t)] ** ((σ[r]-1)/σ[r]) * (Y_fun(r, t) / self.L[(r, s, t)])**(1 / σ[r]))

    def Γ__w_fun(r, t):
        ''' High-low-skilled wage ratio '''
        return(self.w[(r, 'h', t)]/self.w[(r, 'l', t)])

    def c_fun(r, s, b, t):
        ''' Consumption '''
        B = self.B
        if b == 'd':
            B = 0
        return((1-B) * self.w[(r, s, t)] )

    
    def v_fun(r, s, b, t):
        ''' Inner utility '''
        return((1 - τ[r]) * self.c[(r, s, b, t)])

    def mii_fun(r, s, b, t):
        ''' Internal emigrant to stayer ratio '''
        rprime = set(R).difference(r).pop()
        return(((self.v[(rprime, s, 'd', t)]/self.v[(r, s, b, t)])*(1-self.xii[(r, s)]))**(1/μ))

    def mij_fun(r, s, b, t):
        ''' International emigrant to stayer ratio'''
        return(np.array(np.outer(self.v[('n',s,'d',t)],1/self.v[(r,s,b,t)])**(1/μ)\
             * (1-self.xij[(r,s)])**(1/μ) * (1-np.diag(np.ones(len(self.countries))))))    
    
    def Ms_fun(r, s, b, t):
        """ Stayers """
        if b == 'd':
            share = 1-self.ω[(r,t)]
        else:
            share = self.ω[(r,t)]
        return((share * self.N[(r, s, t)])/
               (1 + self.mii[(r, s, b, t)] + self.mij[(r,s,b,t)].sum(1)))
    
    def Mii_fun(r, s, b, t):
        """ Internal emigrants from dry area """
        return(self.mii[(r, s, b, t)] * self.Ms[(r, s, b, t)])

    def Mij_fun(r, s, b, t):
        """ Internal emigrants from dry area """
        return(np.array(self.mij[(r,s,b,t)].multiply(self.Ms[(r,s,b,t)],0)))
    
    def Iii_fun(r,s,t):
        """ Internal immigrant flow"""
        rprime = set(R).difference(r).pop()
        return(self.Mii[(rprime,s,'d',t)] + self.Mii[(rprime,s,'f',t)])

    def Iij_fun(r, s, t):
        ''' International immigrant flow '''
        if r == 'a':
            return(0)
        else:
            return((self.Mij[('a',s,'d',t)] + self.Mij[('a',s,'f',t)] \
                    + self.Mij[('n',s,'d',t)] + self.Mij[('n',s,'f',t)]).sum(0)) #### CHECK THIS AGAIN

    def L_fun(r,s,t):
        rprime = set(R).difference(r).pop()
        L = self.Ms[(r, s, 'd', t)]  +\
            self.Ms[(r, s, 'f', t)]  +\
            self.Iii[(r,s,t)] + self.Iij[(r,s,t)]
        return(L)

    def N_fun(r, s, t):
        ''' Native population '''
        if t < 2040:
            if s == 'l':
                # probabilities to become low-skilled
                p_l = 1 - self.p[(r, 'l', 'd', t-30)]
                p_h = 1 - self.p[(r, 'h', 'd', t-30)]
            else:
                # probabilities to become high-skilled
                p_l = self.p[(r, 'l', 'd', t-30)]
                p_h = self.p[(r, 'h', 'd', t-30)]
            N = self.L[(r, 'l', t-30)] * self.n[(r, 'l', 'd', t-30)] * p_l + \
                self.L[(r, 'h', t-30)] * self.n[(r, 'h', 'd', t-30)] * p_h
        else:
            rprime = set(R).difference(r).pop()
            p_l  = self.p[(r,'h','d',t-30)]
            p_h  = self.p[(r,'h','d',t-30)]
            pf_l = self.p[(r,'l','f',t-30)]
            pf_h = self.p[(r,'h','f',t-30)]
            if s == 'l':
                p_l = 1 - p_l
                pf_l = 1 - pf_l
                p_h = 1 - p_h
                pf_h = 1 - pf_h
            N = self.Ms[(r, 'l', 'd', t-30)]       * self.n[(r, 'l', 'd', t-30)] * p_l  +\
                self.Ms[(r, 'l', 'f', t-30)]       * self.n[(r, 'l', 'f', t-30)] * pf_l +\
                self.Mii[(rprime, 'l', 'd', t-30)] * self.n[(r, 'l', 'd', t-30)] * p_l  +\
                self.Mii[(rprime, 'l', 'f', t-30)] * self.n[(r, 'l', 'f', t-30)] * pf_l +\
                self.Ms[(r, 'h', 'd', t-30)]       * self.n[(r, 'h', 'd', t-30)] * p_l  +\
                self.Ms[(r, 'h', 'f', t-30)]       * self.n[(r, 'h', 'f', t-30)] * pf_l +\
                self.Mii[(rprime, 'h', 'd', t-30)] * self.n[(r, 'h', 'd', t-30)] * p_l  +\
                self.Mii[(rprime, 'h', 'f', t-30)] * self.n[(r, 'h', 'f', t-30)] * pf_l
        return(N)
        
           
    t = 1980
    for r in R:
        self.D[(r, t)] = self.D[(r, t+30)]
        
    """ 1980 Loop """
    def loop1980():
        t = 1980
        for r in R:
            self.Γ__L[(r, t)] = Γ__L_fun(r, t)
            self.Γ__η[(r, t)] = Γ__η_fun(r, t)
        for r in R:
            for s in S:
                self.η[(r, s, t)] = η_fun(r, s, t)
            self.Y[(r, t)] = Y_fun(r, t)
        for r in R:
            for s in S:
                self.w[(r, s, t)] = w_fun(r, s, t)
        for r in R:
            self.Γ__w[(r, t)] = Γ__w_fun(r, t)
        for r in R:
            for s in S:
                for b in BB:
                    self.c[(r, s, b, t)] = c_fun(r, s, b, t)
        for r in R:
            for s in S:
                for b in BB:
                    self.v[(r, s, b, t)] = v_fun(r, s, b, t)
        for r in R:
            for s in S:
                self.N[(r, s, t+30)] = N_fun(r, s, t+30)
                
    """ 2040 Loop """
    def loop2040():
        t = 2040
        for r in R:
            self.Γ__L[(r, t)] = Γ__L_fun(r, t)
            self.Γ__η[(r, t)] = Γ__η_fun(r, t)
        for r in R:
            for s in S:
                self.η[(r, s, t)] = η_fun(r, s, t)
            self.A[(r, t)] = A_fun(r, t)
            self.Y[(r, t)] = Y_fun(r, t)
        for r in R:
            for s in S:
                self.w[(r, s, t)] = w_fun(r, s, t)
        for r in R:
            self.Γ__w[(r, t)] = Γ__w_fun(r, t)
        for r in R:
            for s in S:
                for b in BB:
                    self.c[(r, s, b, t)] = c_fun(r, s, b, t)
        for r in R:
            for s in S:
                for b in BB:
                    self.v[(r, s, b, t)] = v_fun(r, s, b, t)
        for r in R:
            for s in S:
                for b in BB:
                    self.mii[(r, s, b, t)] = mii_fun(r, s, b, t)
        for r in R:
            for s in S:
                for b in BB:
                    self.mij.loc[:,(r, s, b, t)] = mij_fun(r, s, b, t)
                
  
    
    """ 2010-2040 Loop """
    def loop2010to2100():
        plt.figure(figsize=(15, 4))
        Ts = [2010, 2040, 2070, 2100]
        for t in Ts:
            self.ΔL = 1
            count = 0
            conv = 0.8
            
            # Guess w & L
            for r in R:
                for s in S:
                    if t > 2010:
                        self.L[(r, s, t)] = self.L[(r, s, t-30)]

            Δ = self.ΔL**2
            self.Δ = []

            while (Δ > tol) & (count < max_iter):
                if count > 0:
                    self.L[(r, s, t)] = conv * self.L[(r, s, t)] +\
                    (1-conv) * self.L_old[(r, s)]

                self.L_old = self.L.xs(t, 1, 2)
                
                for r in R:
                    self.Γ__L[(r, t)] = Γ__L_fun(r, t)
                    self.Γ__η[(r, t)] = Γ__η_fun(r, t)
                for r in R:
                    for s in S:
                        self.η[(r, s, t)] = η_fun(r, s, t)
                    self.A[(r, t)] = A_fun(r, t)
                    self.Y[(r, t)] = Y_fun(r, t)
                for r in R:
                    for s in S:
                        self.w[(r, s, t)] = w_fun(r, s, t)
                for r in R:
                    self.Γ__w[(r, t)] = Γ__w_fun(r, t)
                    for s in S:
                        for b in BB:
                            self.c[(r, s, b, t)] = c_fun(r, s, b, t)
                            self.v[(r, s, b, t)] = v_fun(r, s, b, t)
                for r in R:
                    for s in S:
                        for b in BB:
                            self.mii[(r, s, b, t)] = mii_fun(r, s, b, t)
                for r in R:
                    for s in S:
                        for b in BB:
                            self.mij.loc[:,(r, s, b, t)] = mij_fun(r, s, b, t)
                for r in R:
                    for s in S:
                        for b in BB:
                            self.Ms[(r, s, b, t)] = Ms_fun(r, s, b, t)
                            self.Mii[(r, s, b, t)] = Mii_fun(r, s, b, t)
                            self.Mij[(r, s, b, t)] = Mij_fun(r, s, b, t)
                for r in R:
                    for s in S:
                        self.Iii[(r,s,t)] = Iii_fun(r,s,t)
                        self.Iij[(r,s,t)] = Iij_fun(r,s,t)
                for r in R:
                    for s in S:
                        self.L[(r, s, t)] = L_fun(r, s, t)
                for r in R:
                    for s in S:
                        self.N[(r, s, t+30)] = N_fun(r, s, t+30)
                        
                
                self.ΔL = (self.L_old - self.L.xs(t, 1, 2)).sum().sum()
                Δ = self.ΔL**2
                self.Δ.append(Δ)
                
                count += 1
               
                
            if report == True:
                plt.subplot(1, len(Ts), int((t-Ts[0])/30+1))
                plt.plot(self.Δ[report_from:max_iter])
                plt.title('Year = '+str(t)
                         +'\n'+'N° of Iterations = '+str(count)
                         +'\n'+'ΔL = '+str(self.ΔL))
                plt.grid(True)
                plt.xlabel('iteration i')
                plt.ylabel('$ΔL_i^2$')
                plt.yscale('log')
    plt.show()   
    
    """ 2010-2040 Loop """
    def loop2010to2100_new():
        plt.figure(figsize=(15, 4))
        conv = 0.8 # convergence parameter
        
        Ts = [2010, 2040, 2070, 2100]
        for t in Ts:
            count = 0 # counter
            self.ΔL = 1
            
            for r in R:
                for s in S:
                    # initial guess for L(r,s,t) = L(r,s,t-1)
                    self.L[(r, s, t)] = self.L[(r, s, t-30)] 
            
            # quadratic differentials in L
            Δ = self.ΔL**2
            # create vector to track evolution of quadratic differentials
            self.Δ = [] 

            while (Δ > tol) & (count < max_iter):
                if count > 0:
                    # after first run, L(r,s,t) = convex combination of prediction & guess
                    self.L[(r, s, t)] = conv * self.L[(r, s, t)] + (1-conv) * self.L_old[(r, s)]

                # track old value of L
                self.L_old = self.L.xs(t, 1, 2)
                
                for r in R:
                    self.Γ__L[(r, t)] = Γ__L_fun(r, t)
                    self.Γ__η[(r, t)] = Γ__η_fun(r, t)
                for r in R:
                    for s in S:
                        self.η[(r, s, t)] = η_fun(r, s, t)
                    self.A[(r, t)] = A_fun(r, t)
                    self.Y[(r, t)] = Y_fun(r, t)
                for r in R:
                    for s in S:
                        self.w[(r, s, t)] = w_fun(r, s, t)
                for r in R:
                    self.Γ__w[(r, t)] = Γ__w_fun(r, t)
                    for s in S:
                        for b in BB:
                            self.c[(r, s, b, t)] = c_fun(r, s, b, t)
                            self.v[(r, s, b, t)] = v_fun(r, s, b, t)
                for r in R:
                    for s in S:
                        for b in BB:
                            self.mii[(r, s, b, t)] = mii_fun(r, s, b, t)
                for r in R:
                    for s in S:
                        for b in BB:
                            self.mij.loc[:,(r, s, b, t)] = mij_fun(r, s, b, t)
                for r in R:
                    for s in S:
                        for b in BB:
                            self.Ms[(r, s, b, t)] = Ms_fun(r, s, b, t)
                            self.Mii[(r, s, b, t)] = Mii_fun(r, s, b, t)
                            self.Mij[(r, s, b, t)] = Mij_fun(r, s, b, t)
                for r in R:
                    for s in S:
                        self.Iii[(r,s,t)] = Iii_fun(r, s, t)
                        self.Iij[(r,s,t)] = Iij_fun(r, s, t)
                for r in R:
                    for s in S:
                        self.L[(r, s, t)] = L_fun(r, s, t)
                for r in R:
                    for s in S:
                        self.N[(r, s, t+30)] = N_fun(r, s, t+30)
                        
                
                self.ΔL = ((self.L_old - self.L.xs(t, 1, 2))**2).sum().sum()
                Δ = self.ΔL
                self.Δ.append(Δ)
                
                count += 1
               
                
            if report == True:
                plt.subplot(1, len(Ts), int((t-Ts[0])/30+1))
                plt.plot(self.Δ[report_from:max_iter])
                plt.title('Year = '+str(t)
                         +'\n'+'N° of Iterations = '+str(count)
                         +'\n'+'ΔL = '+str(self.ΔL))
                plt.grid(True)
                plt.xlabel('iteration i')
                plt.ylabel('$ΔL_i^2$')
                plt.yscale('log')
    plt.show()
    
    loop1980()
    #loop2040()
    #loop2010to2100()
    loop2010to2100_new()
    
model.simulate = simulate

## Checks

### Native population

In [ ]:
def native_split(self,r,s,t):

    path = 'population_splits/'
    fig = plt.figure(figsize=(16,26))

    split = pd.DataFrame({'stayers_d' : (self.Ms[(r,s,'d',t)]/self.N[(r,s,t)]),
                          'stayers_f' : (self.Ms[(r,s,'f',t)]/self.N[(r,s,t)]),
                          'internal_emigrants_d': (self.Mii[(r,s,'d',t)]/self.N[(r,s,t)]),
                          'internal_emigrants_f': (self.Mii[(r,s,'f',t)]/self.N[(r,s,t)]),
                          'internat_emigrants_d': (self.Mij[(r,s,'d',t)].sum(1)/self.N[(r,s,t)]),
                          'internat_emigrants_f': (self.Mij[(r,s,'f',t)].sum(1)/self.N[(r,s,t)]),
                          })

    
    split = split.sort_values(by=list(split))

    
    hatch = ['/']
    for i in range(len(split)):
        hatch.append(hatch[-1]+'/')
    hatch =  itertools.cycle(hatch)
    left = 0
    for i in list(split):
        plt.barh(split.index,split[i],left=left,label=i,hatch=next(hatch),edgecolor='white')
        left = left + split[i]
    ticks = [0,.25,.5,.75,1]
    plt.tick_params(axis='both', labelsize=6)
    plt.xticks(ticks)
    for x in ticks:
        if 0 < x < 1: 
            plt.axvline(x,c='k',linestyle='--')
    plt.axvline(1,c='k')
    #plt.tight_layout()
    plt.legend(loc=9,ncol=3)
    plt.title('Native population split in '+r+s+str(t))
    plt.show()
    fig.savefig(path+'N'+str(t)+r+s+'.pdf',dpi=300)
    
model.native_split = native_split

### Resident population

In [ ]:
def resident_split(self,r,s,t):
    path = 'population_splits/'
    fig = plt.figure(figsize=(16,26))
    rprime = set(R).difference(r).pop()

    split = pd.DataFrame({'stayers_d' : (self.Ms[(r,s,'d',t)])/self.L[(r,s,t)],
                          'stayers_f' : (self.Ms[(r,s,'f',t)])/self.L[(r,s,t)],
                          'internal_immigrants_d': (self.Mii[(rprime,s,'d',t)])/self.L[(r,s,t)],
                          'internal_immigrants_f': (self.Mii[(rprime,s,'f',t)]/self.L[(r,s,t)]),
                          'internat_immigrants_a_d': (self.Mij[('a',s,'d',t)].sum(0))/self.L[(r,s,t)],
                          'internat_immigrants_a_f': (self.Mij[('a',s,'f',t)].sum(0))/self.L[(r,s,t)],
                          'internat_immigrants_n_d': (self.Mij[('n',s,'d',t)].sum(0))/self.L[(r,s,t)],
                          'internat_immigrants_n_f': (self.Mij[('n',s,'f',t)].sum(0))/self.L[(r,s,t)],
                          })


    if r == 'a':
        split['internat_immigrants_a_d'] = 0
        split['internat_immigrants_n_d'] = 0
        split['internat_immigrants_a_f'] = 0
        split['internat_immigrants_n_f'] = 0
        
    split = split.sort_values(by=list(split))
    
    hatch = ['/']
    for i in range(len(split)):
        hatch.append(hatch[-1]+'/')
    hatch =  itertools.cycle(hatch)
    left = 0
    for i in list(split):
        plt.barh(split.index,split[i],left=left,label=i,hatch=next(hatch),edgecolor='white')
        left = left + split[i]
    ticks = [0,.25,.5,.75,1]
    plt.tick_params(axis='both', labelsize=6)
    plt.xticks(ticks)
    for x in ticks:
        if 0 < x < 1: 
            plt.axvline(x,c='k',linestyle='--')
    plt.axvline(1,c='k')
    #plt.tight_layout()
    plt.legend(loc=9,ncol=4)
    plt.title('Resident population composition in '+r+s+str(t))
    plt.show()
    fig.savefig(path+'L'+str(t)+r+s+'.pdf',dpi=300)

model.resident_split = resident_split

### Further checks

In [ ]:
def checks(self):
    YminusLw = []
    NminusL = []
    for t in T:
        # Total production = total income
        YminusLw.append((self.w.xs(t,1,2)*self.L.xs(t,1,2)).sum().sum() - self.Y.xs(t,1,1).sum().sum())
        # Total native population = total resident population
        NminusL.append((self.N.xs(t,1,2)-self.L.xs(t,1,2)).sum().sum())
        
    print('YminusLw',YminusLw)
    print('NminusL',NminusL)
    
model.checks = checks

## Save output

In [ ]:
def save_output(self):
    """ Saves output of simulation """

    self.var = {
    'irt':   ['A', 'D', 'Y', 'xii', 'Γ__L', 'Γ__n', 'Γ__w', 'Γ__η', 'ω'],
    'irst':  ['Iii', 'Iij', 'L', 'N', 'w', 'xij', 'η'],
    'irsbt': ['Mii', 'Ms', 'c', 'mii', 'n', 'p', 'v'],
    'ijrsbt': ['Mij','mij'],
    }

    def output(dim):
        var_data = [s1.__dict__[v].dropna(axis=1, how='all') for v in self.var[dim]]
        return(pd.concat(var_data, keys=self.var[dim], axis=1))

    writer = pd.ExcelWriter(
        'output/'+self.name+'_'+str(datetime.datetime.now())[:10]+'.xlsx', engine='xlsxwriter')

    for dim in self.var.keys():
        output(dim).to_excel(writer, sheet_name=dim)
        output(dim).describe().T.to_excel(
            writer, sheet_name=dim+'_statistics')
    writer.save()
    
model.save_output = save_output

# Do Simulations


## Simulation 1: Bilateral migration

In [ ]:
s1 = model(name='intermediate', tol=1e-5, scenario='intermediate')
s1.correct_south_south()
s1.calib_epsilon()
s1.calib_kappa()
s1.calib_migcosts()
s1.fertility()
s1.simulate(report=True, report_from=0, max_iter=50)
s1.save_output()

In [ ]:
s2 = model(name='minimalist', tol=1e-5, scenario='minimalist')
s2.correct_south_south()
s2.calib_epsilon()
s2.calib_kappa()
s2.calib_migcosts()
s2.fertility()
s2.simulate(report=True, report_from=0, max_iter=50)
s2.save_output()

In [ ]:
s3 = model(name='maximalist', tol=1e-5, scenario='maximalist')
s3.correct_south_south()
s3.calib_epsilon()
s3.calib_kappa()
s3.calib_migcosts()
s3.fertility()
s3.simulate(report=True, report_from=0, max_iter=50)
s3.save_output()

In [ ]:
s2 = model(name='simulation2', tol=1e-5)
s2.correct_south_south()

In [ ]:
t = 2100
a = s1.L.xs(t,1,2).sum().sum()
b = s1.N.xs(t,1,2,False).sum().sum()
print(a)
print(b)
print(a-b)

In [ ]:
(s1.L.xs(2010,1,2)-s2.L.xs(2010,1,2)).head()

In [ ]:
s2.L.xs(2010,1,2).head()

In [ ]:
a = s1.L.xs(2010,1,2).sum().sum()
b = s1.N.xs(2010,1,2,False).sum().sum()
print(a)
print(b)
print(a-b)

In [ ]:
a = s1.L.xs(2010,1,2).sum().sum()
b = s1.N.xs(2010,1,2,False).sum().sum()
print(a)
print(b)
print(a-b)

In [ ]:
a = s1.L.xs(2010,1,2).sum().sum()
b = s1.N.xs(2010,1,2,False).sum().sum()
print(a)
print(b)
print(a-b)

In [ ]:
s1.N.sum().sum()

In [ ]:
sx = s1
for t in T[1:]:
    for r in R:
        for s in S:
            sx.native_split(r,s,t)            
    for r in R:
        for s in S:
            sx.resident_split(r,s,t)


In [ ]:
for r in R:
    for s in S: 
        fig = plt.figure(figsize=(5,5))
        sns.regplot(s1.n[(r,s,'d',2010)],s1.n[(r,s,'d',2010)],order=1)
        plt.title(r+s)
        plt.xlabel('n')
        plt.ylabel('p')
        plt.show()

## Simulation 2: Migration to the OECD

In [ ]:
s2 = simulation(name='simulation2', tol=1e-5)
s2.calib_epsilon()
s2.calib_kappa()
t = 2010
for r in R:
    for s in S:
        s2.Mij[(r,s,'d',t)] = s2.Mij[(r,s,'d',t)].multiply(s2.OECD,1)
s2.calib_migcosts()
s2.calib_pop_growth()
s2.simulate(report=True, report_from=0, max_iter=50,fert_new=False,calib_fert=True)


In [ ]:
for sx in [s1,s2]:
    print('s1.name')
    for t in T[1:]:
        for r in R:
            for s in S:
                sx.native_split(r,s,t)            
        for r in R:
            for s in S:
                sx.resident_split(r,s,t)
    

In [ ]:
check.L.xs(2010,1,2)-s1.L.xs(2010,1,2)

In [ ]:
check = pd.read_excel(
        'checks/check_raw.xlsx', sheet_name='irst', index_col=0, header=list(range(len('irst'))))

def country_name(x):
    return(pycountry.countries.indices['alpha3'][x].name)

def plot_scatters(num=7,size=10):
    tol = 0
    markers = ['o','^','s','x']
    for t in [2010,2040, 2070,2100]:
        print(t)
        print('------------------------------------')
        for var in ['w','L','N']:
            plt.figure(figsize=(size, size))
            count = 1
            for r in R:
                for s in S:
                    if var != 'η':
                        conditions = pd.Series(True, index=s1.countries)
                        y = s1.__dict__[var][(r, s, t)][conditions]
                        #x = s2.__dict__[var][(r, s, t)][conditions]
                        x = check[(var, r, s, t)][conditions]

                        plt.subplot(2, 2, count)
                        sns.regplot(
                            x, y, label=r+s,fit_reg=False,scatter=True, scatter_kws={'s': 25, 'alpha': 0.7}, marker=markers[count-1], color=colors[count-1])
                        for i in abs((((y+s1.tol)/(x+s1.tol))-1)).nlargest(num).index:
                            plt.annotate(s1.iso[i],xy = [x[i],y[i]],rotation=45,weight="bold")

                        plt.legend()
                        lims = [min(x.append(y)), max(x.append(y))]
                        plt.xlim(lims)
                        plt.ylim(lims)
                        z = np.linspace(lims[0],lims[1])
                        plt.plot(z,z,color='k')
                        plt.xlabel('Excel')
                        plt.ylabel('Code')
                        plt.xscale('log')
                        plt.yscale('log')
                        plt.tight_layout()
                        plt.title('$'+var+'_{'+r+','+s+','+str(t)+'}$'+
                                 '\n Correlation = '+str(round(np.corrcoef(x, y)[0,1],4)))
                        count += 1
            plt.show()

In [ ]:
plot_scatters(size=10)

# References

(<a id="cit-Burzynski2018" href="#call-Burzynski2018">Burzynski, Deuster <em>et al.</em>, 2018</a>) Burzynski Michal, Deuster Christoph, Docquier Frederic <em>et al.</em>, ``_Climate change, Inequality and Migration_'', Working paper, vol. , number , pp. , January 2018.  [online](https://perso.uclouvain.be/frederic.docquier/filePDF/BDDD_Nova2018.pdf)

